In [1]:
res = {
    "class_instance": None
}

def class_decorator(cls):
   for name, method in cls.__dict__.iteritems():
        if hasattr(method, "use_class"):
            # do something with the method and class
            res["class_instance"] = cls
#             print(name, cls)
   return cls

def method_decorator(view):
    # mark the method as something that requires view's class
    view.use_class = True
    return view

@class_decorator
class ModelA(object):
    @method_decorator
    def a_method(self):
        # do some stuff
        print("a_method")
        pass
    
obj = ModelA()
print(res["class_instance"] == obj.__class__)
from pydoc import locate
a = locate("toto")
print(a)

True
None


In [19]:
from protocol0.utils.utils import _has_callback_queue, is_lambda, get_callable_repr, nop, get_class_name_from_method
from protocol0.sequence.Sequence import Sequence
from protocol0 import EmptyModule, create_instance
from protocol0.utils.callback_descriptor import CallableWithCallbacks
from pydoc import classname
from qualname import qualname

p0 = create_instance(EmptyModule(is_false=False))

def titi_listener():
    pass

class Test():
    def toto():
        pass
    
seq = Sequence()
# print(get_callable_repr(seq.add))
cwc = CallableWithCallbacks(titi_listener, True)

seq.add(Test().toto, complete_on=cwc)
print(get_callable_repr(cwc))
print(get_callable_repr(seq._steps[0]))

def print_info(obj):
    print(get_class_name_from_method(obj))
    print(classname(obj, ""))
    print(qualname(obj))
    
    
# print("---")
# print_info(titi_listener)
# print_info(Test().toto)

unknown <function titi_listener at 0x069AC8B0>.titi_listener (cwc 111052016)
[step Test.toto (and wait for listener call : unknown <function titi_listener at 0x069AC8B0>.titi_listener (cwc 111052016))]


In [18]:
from protocol0.lom.clip_slot.ClipSlot import ClipSlot

cs = ClipSlot.make(object(), object())

SubjectSlotError: Subject <object object at 0x04EF4BF0> missing "add" method for event: has_clip